In [1]:
import json

class Nodo:
    def __init__(self, data=None):
        self.next = None
        self.data = data
        self.bef = None

class ColaDob:
    def __init__(self, limit=9):
        self.limite = limit
        self.front = None
        self.rear = None
        self.size = 0

    def isEmpty(self):
        return self.size <= 0

    def isFull(self):
        return self.size >= self.limite

    def enColarD(self, data):
        nodo = Nodo(data)
        if self.isFull():
            print("Cola llena")
            return

        if self.isEmpty():
            self.front = self.rear = nodo
        else:
            nodo.bef = self.rear
            self.rear.next = nodo
            self.rear = nodo
        self.size += 1
        return data

    def desenColarD(self):
        if self.isEmpty():
            print("Cola vacía")
            return None
        nodo = self.front
        self.front = self.front.next
        if self.front is None:
            self.rear = None
        else:
            self.front.bef = None
        self.size -= 1
        return nodo.data

    def imprimirColaD(self):
        valor = self.front
        print("Cola: ", end="")
        while valor:
            self.imprimirNodo(valor.data)
            valor = valor.next
        print("None")

    def imprimirColaIz(self):
        valor = self.rear
        print("Cola: ", end="")
        while valor:
            self.imprimirNodo(valor.data)
            valor = valor.bef
        print("None")

    def imprimirNodo(self, data):
        if isinstance(data, dict):
            formatted_data = ', '.join(f"{k}: {v}" for k, v in data.items())
            print(formatted_data, end=" => ")
        else:
            print(data, end=" => ")

    def captarColaS(self, ColaS):
        if ColaS.isEmpty():
            return
        else:
            self.enColarD(ColaS.desenColar())
            self.captarColaS(ColaS)


cola = ColaDob()


try:
    with open('inventario.txt', 'r') as file:
        for line in file:
            data = json.loads(line.strip())
            cola.enColarD(data)
except FileNotFoundError:
    print("El archivo 'inventario1.txt' no se encontró. Asegúrate de que el archivo esté en el mismo directorio que tu código.")

# Imprimir la cola
cola.imprimirColaD()

print('///////////////')
cola.imprimirColaIz()


Cola: id: 1, nombre: Manzana, tipo: perecedero, categoria: fruta, cantidad: 50, fech_venc: 2024-10-26 => id: 2, nombre: Arroz, tipo: no perecedero, categoria: grano, cantidad: 100, fech_venc: 2030-01-20 => id: 3, nombre: Leche, tipo: perecedero, categoria: lacteo, cantidad: 30, fech_venc: 2025-11-10 => id: 4, nombre: Lentejas, tipo: no perecedero, categoria: grano, cantidad: 80, fech_venc: 2024-10-26 => id: 5, nombre: Pollo, tipo: perecedero, categoria: carnes, cantidad: 20, fech_venc: 2024-10-27 => id: 6, nombre: Pasta, tipo: no perecedero, categoria: pasta, cantidad: 60, fech_venc: 2027-12-12 => id: 7, nombre: Platano, tipo: perecedero, categoria: fruta, cantidad: 55, fech_venc: 2025-01-12 => id: 8, nombre: Naranja, tipo: perecedero, categoria: fruta, cantidad: 2, fech_venc: 2024-11-15 => None
///////////////
Cola: id: 8, nombre: Naranja, tipo: perecedero, categoria: fruta, cantidad: 2, fech_venc: 2024-11-15 => id: 7, nombre: Platano, tipo: perecedero, categoria: fruta, cantidad: 55,

In [14]:
import tkinter as tk
from tkinter import ttk, messagebox
import json
from datetime import datetime

class AlmacenGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistema de Gestión de Almacenes")
        self.root.geometry("800x600")
        
        # Cargar datos iniciales
        self.cargar_datos()
        
        # Crear pestañas
        self.notebook = ttk.Notebook(root)
        self.notebook.pack(expand=True, fill='both', padx=10, pady=5)
        
        # Pestaña de Visualización
        self.tab_visualizar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_visualizar, text='Visualizar Inventario')
        
        # Pestaña de Inserción
        self.tab_insertar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_insertar, text='Insertar Producto')
        
        # Pestaña de Búsqueda
        self.tab_buscar = ttk.Frame(self.notebook)
        self.notebook.add(self.tab_buscar, text='Buscar Producto')
        
        self.setup_visualizar_tab()
        self.setup_insertar_tab()
        self.setup_buscar_tab()
        
    def cargar_datos(self):
        try:
            with open('inventario.txt', 'r') as file:
                contenido = file.read()
                # Convertir el contenido en una lista de diccionarios
                self.inventario = []
                for item in contenido.strip().split('}'):
                    if item.strip():
                        # Asegurarse de que el item termine con }
                        if not item.endswith('}'):
                            item += '}'
                        # Limpiar el item de espacios y caracteres no deseados
                        item = item.strip().lstrip('{').strip()
                        if item:
                            try:
                                self.inventario.append(json.loads('{' + item))
                            except json.JSONDecodeError:
                                continue
        except FileNotFoundError:
            self.inventario = []
            messagebox.showwarning("Advertencia", "No se encontró el archivo inventarios1.txt")
    
    def guardar_datos(self):
        with open('inventario.txt', 'w') as file:
            for item in self.inventario:
                file.write(str(item))
    
    def setup_visualizar_tab(self):
        # Crear Treeview
        columns = ('ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree = ttk.Treeview(self.tab_visualizar, columns=columns, show='headings')
        
        # Configurar columnas
        for col in columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, width=100)
        
        # Agregar scrollbar
        scrollbar = ttk.Scrollbar(self.tab_visualizar, orient='vertical', command=self.tree.yview)
        self.tree.configure(yscrollcommand=scrollbar.set)
        
        # Posicionar elementos
        self.tree.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        
        # Cargar datos en el Treeview
        self.actualizar_treeview()
        
    def actualizar_treeview(self):
        # Limpiar Treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Insertar datos
        for producto in self.inventario:
            self.tree.insert('', 'end', values=(
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))
    
    def setup_insertar_tab(self):
        # Crear formulario
        ttk.Label(self.tab_insertar, text="ID:").grid(row=0, column=0, padx=5, pady=5)
        self.id_entry = ttk.Entry(self.tab_insertar)
        self.id_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Nombre:").grid(row=1, column=0, padx=5, pady=5)
        self.nombre_entry = ttk.Entry(self.tab_insertar)
        self.nombre_entry.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Tipo:").grid(row=2, column=0, padx=5, pady=5)
        self.tipo_combo = ttk.Combobox(self.tab_insertar, values=['perecedero', 'no perecedero'])
        self.tipo_combo.grid(row=2, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Cantidad:").grid(row=3, column=0, padx=5, pady=5)
        self.cantidad_entry = ttk.Entry(self.tab_insertar)
        self.cantidad_entry.grid(row=3, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_insertar, text="Fecha Vencimiento:").grid(row=4, column=0, padx=5, pady=5)
        self.fecha_entry = ttk.Entry(self.tab_insertar)
        self.fecha_entry.grid(row=4, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_insertar, text="Insertar Producto", command=self.insertar_producto).grid(row=5, column=0, columnspan=2, pady=20)
    
    def insertar_producto(self):
        try:
            nuevo_producto = {
                'id': int(self.id_entry.get()),
                'nombre': self.nombre_entry.get(),
                'tipo': self.tipo_combo.get(),
                'cantidad': int(self.cantidad_entry.get()),
                'fech_venc': self.fecha_entry.get()
            }
            
            # Validaciones básicas
            if not nuevo_producto['nombre'] or not nuevo_producto['tipo']:
                messagebox.showerror("Error", "Todos los campos son obligatorios")
                return
            
            # Verificar si el ID ya existe
            if any(p['id'] == nuevo_producto['id'] for p in self.inventario):
                messagebox.showerror("Error", "El ID ya existe")
                return
            
            self.inventario.append(nuevo_producto)
            self.guardar_datos()
            self.actualizar_treeview()
            
            # Limpiar campos
            self.id_entry.delete(0, 'end')
            self.nombre_entry.delete(0, 'end')
            self.tipo_combo.set('')
            self.cantidad_entry.delete(0, 'end')
            self.fecha_entry.delete(0, 'end')
            
            messagebox.showinfo("Éxito", "Producto insertado correctamente")
            
        except ValueError:
            messagebox.showerror("Error", "Por favor, ingrese valores válidos")
    
    def setup_buscar_tab(self):
        # Crear campos de búsqueda
        ttk.Label(self.tab_buscar, text="Buscar por nombre:").grid(row=0, column=0, padx=5, pady=5)
        self.buscar_nombre_entry = ttk.Entry(self.tab_buscar)
        self.buscar_nombre_entry.grid(row=0, column=1, padx=5, pady=5)
        
        ttk.Label(self.tab_buscar, text="Buscar por tipo:").grid(row=1, column=0, padx=5, pady=5)
        self.buscar_tipo_combo = ttk.Combobox(self.tab_buscar, values=['perecedero', 'no perecedero'])
        self.buscar_tipo_combo.grid(row=1, column=1, padx=5, pady=5)
        
        ttk.Button(self.tab_buscar, text="Buscar", command=self.buscar_productos).grid(row=2, column=0, columnspan=2, pady=20)
        
        # Crear Treeview para resultados
        columns = ('ID', 'Nombre', 'Tipo', 'Cantidad', 'Fecha Vencimiento')
        self.tree_busqueda = ttk.Treeview(self.tab_buscar, columns=columns, show='headings')
        
        for col in columns:
            self.tree_busqueda.heading(col, text=col)
            self.tree_busqueda.column(col, width=100)
        
        self.tree_busqueda.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
    
    def buscar_productos(self):
        # Limpiar resultados anteriores
        for item in self.tree_busqueda.get_children():
            self.tree_busqueda.delete(item)
        
        nombre_buscar = self.buscar_nombre_entry.get().lower()
        tipo_buscar = self.buscar_tipo_combo.get()
        
        resultados = []
        for producto in self.inventario:
            if ((not nombre_buscar or nombre_buscar in producto['nombre'].lower()) and
                (not tipo_buscar or tipo_buscar == producto['tipo'])):
                resultados.append(producto)
        
        # Mostrar resultados
        for producto in resultados:
            self.tree_busqueda.insert('', 'end', values=(
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))
        
        if not resultados:
            messagebox.showinfo("Búsqueda", "No se encontraron productos")

def buscar_productos(self):
    for item in self.tree_busqueda.get_children():
        self.tree_busqueda.delete(item)
        
    codigo_buscar = self.buscar_codigo_entry.get()
    nombre_buscar = self.buscar_nombre_entry.get().lower()
    tipo_buscar = self.buscar_tipo_combo.get()
    
    if codigo_buscar:
        # Búsqueda por código usando BST
        nodo = self.arbol.buscar(codigo_buscar)
        if nodo:
            self.tree_busqueda.insert('', 'end', values=(
                nodo.producto['codigo'],
                nodo.producto['id'],
                nodo.producto['nombre'],
                nodo.producto['tipo'],
                nodo.producto['cantidad'],
                nodo.producto['fech_venc']
            ))
    else:
        # Búsqueda por nombre/tipo usando la lista
        resultados = []
        for producto in self.inventario:
            if ((not nombre_buscar or nombre_buscar in producto['nombre'].lower()) and
                (not tipo_buscar or tipo_buscar == producto['tipo'])):
                resultados.append(producto)
        
        # Mostrar resultados
        for producto in resultados:
            self.tree_busqueda.insert('', 'end', values=(
                producto['codigo'],
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))
        
        if not resultados:
            messagebox.showinfo("Búsqueda", "No se encontraron productos")

def actualizar_treeview(self):
    # Limpiar Treeview
    for item in self.tree.get_children():
        self.tree.delete(item)
        
    # Insertar datos ordenados por código
    productos_ordenados = self.arbol.inorder()
    for producto in productos_ordenados:
        self.tree.insert('', 'end', values=(
            producto['codigo'],
            producto['id'],
            producto['nombre'],
            producto['tipo'],
            producto['cantidad'],
            producto['fech_venc']
        ))

def guardar_datos(self):
    with open('inventario.txt', 'w') as file:
        for producto in self.inventario:
            file.write(str(producto))

if __name__ == '__main__':
    root = tk.Tk()
    app = AlmacenGUI(root)
    root.mainloop()

KeyError: 'id'

HI THERE!!

In [13]:
def buscar_productos(self):
    for item in self.tree_busqueda.get_children():
        self.tree_busqueda.delete(item)
        
    codigo_buscar = self.buscar_codigo_entry.get()
    nombre_buscar = self.buscar_nombre_entry.get().lower()
    tipo_buscar = self.buscar_tipo_combo.get()
    
    if codigo_buscar:
        # Búsqueda por código usando BST
        nodo = self.arbol.buscar(codigo_buscar)
        if nodo:
            self.tree_busqueda.insert('', 'end', values=(
                nodo.producto['codigo'],
                nodo.producto['id'],
                nodo.producto['nombre'],
                nodo.producto['tipo'],
                nodo.producto['cantidad'],
                nodo.producto['fech_venc']
            ))
    else:
        # Búsqueda por nombre/tipo usando la lista
        resultados = []
        for producto in self.inventario:
            if ((not nombre_buscar or nombre_buscar in producto['nombre'].lower()) and
                (not tipo_buscar or tipo_buscar == producto['tipo'])):
                resultados.append(producto)
        
        # Mostrar resultados
        for producto in resultados:
            self.tree_busqueda.insert('', 'end', values=(
                producto['codigo'],
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))
        
        if not resultados:
            messagebox.showinfo("Búsqueda", "No se encontraron productos")

    def actualizar_treeview(self):
        # Limpiar Treeview
        for item in self.tree.get_children():
            self.tree.delete(item)
        
        # Insertar datos ordenados por código
        productos_ordenados = self.arbol.inorder()
        for producto in productos_ordenados:
            self.tree.insert('', 'end', values=(
                producto['codigo'],
                producto['id'],
                producto['nombre'],
                producto['tipo'],
                producto['cantidad'],
                producto['fech_venc']
            ))

    def guardar_datos(self):
        with open('inventarios1.txt', 'w') as file:
            for producto in self.inventario:
                file.write(str(producto))

if __name__ == '__main__':
    root = tk.Tk()
    app = AlmacenGUI(root)
    root.mainloop()